In [20]:
import pandas as pd
import json

In [21]:
import os
if 'R_HOME' not in os.environ:
  os.environ['R_HOME'] = '/usr/lib/R'

In [22]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
def convert_list_of_dicts_into_dataframe(dict_list):
  output = {}
  for keyname in dict_list[0].keys():
    output[keyname] = []
  for item in dict_list:
    for k,v in item.items():
      output[k].append(v)
  return pd.DataFrame.from_dict(output)

#print(convert_list_of_dicts_into_dataframe([
#  {'a': 3, 'b': 5},
#  {'a': 4, 'b': 6}
#]))

In [24]:
sessions_for_user_by_day_and_goal_for_all_users = json.load(open('browser_all_session_info_sept18.json'))

In [108]:
def print_data_for_users():
  for sessions_for_user_by_day_and_goal in sessions_for_user_by_day_and_goal_for_all_users:
    user = sessions_for_user_by_day_and_goal['user']
    if user == '6216c587462fb5fa9ca9f0e9':
      print(sessions_for_user_by_day_and_goal)
print_data_for_users()

{'user': '6216c587462fb5fa9ca9f0e9', 'days_domains_and_sessions': [{'epoch': 954, 'domains_and_sessions': [{'domain': 'www.playpcesor.com', 'time_on_domain_today': 105384637, 'time_on_other_goal_domains_today': 613, 'is_goal_enabled': False, 'is_goal_frequent': False, 'session_info_list_for_domain': [{'domain': 'www.playpcesor.com', 'is_goal_enabled': False, 'is_goal_frequent': False, 'intervention_active': None, 'duration': 30, 'local_timestamp': 1534137769222.0, 'local_epoch': 954}]}, {'domain': 'mega.nz', 'time_on_domain_today': 105384637, 'time_on_other_goal_domains_today': 613, 'is_goal_enabled': False, 'is_goal_frequent': False, 'session_info_list_for_domain': [{'domain': 'mega.nz', 'is_goal_enabled': False, 'is_goal_frequent': False, 'intervention_active': None, 'duration': 26, 'local_timestamp': 1534138779328.0, 'local_epoch': 954}]}, {'domain': 'www.google.com.tw', 'time_on_domain_today': 105384637, 'time_on_other_goal_domains_today': 613, 'is_goal_enabled': False, 'is_goal_fr

In [99]:
def passes_filters(filter_funcs, item):
  for filter_func in filter_funcs:
    if not filter_func(item):
      return False
  return True

def make_dictlist_days(sessions_for_user_by_day_and_goal_for_all_users, item_filter_funcs=[], day_filter_funcs=[], user_filter_funcs=[]):
  output = []
  for sessions_for_user_by_day_and_goal in sessions_for_user_by_day_and_goal_for_all_users:
    user = sessions_for_user_by_day_and_goal['user']
    if not passes_filters(user_filter_funcs, sessions_for_user_by_day_and_goal):
      continue
    epoch_list = [x['epoch'] for x in sessions_for_user_by_day_and_goal['days_domains_and_sessions']]
    first_epoch_for_user = min(epoch_list)
    last_epoch_for_user = max(epoch_list)
    for day_domains_and_sessions in sessions_for_user_by_day_and_goal['days_domains_and_sessions']:
      epoch = day_domains_and_sessions['epoch']
      day_domains_and_sessions['days_since_install'] = epoch - first_epoch_for_user
      day_domains_and_sessions['days_until_last'] = last_epoch_for_user - epoch
      if not passes_filters(day_filter_funcs, day_domains_and_sessions):
        continue
      for domain_and_sessions in day_domains_and_sessions['domains_and_sessions']:
        domain = domain_and_sessions['domain']
        time_on_domain_today = domain_and_sessions['time_on_domain_today']
        time_on_other_goal_domains_today = domain_and_sessions['time_on_other_goal_domains_today']
        is_goal_enabled = domain_and_sessions['is_goal_enabled']
        is_goal_frequent = domain_and_sessions['is_goal_frequent']
        newitem = {
          'user': user,
          'epoch': epoch,
          'domain': domain,
          'time_on_domain_today': time_on_domain_today,
          'time_on_other_goal_domains_today': time_on_other_goal_domains_today,
          'is_goal_enabled': is_goal_enabled,
          'is_goal_frequent': is_goal_frequent,
        }
        if not passes_filters(item_filter_funcs, newitem):
          continue
        output.append(newitem)
  return output

def make_dataframe_days(sessions_for_user_by_day_and_goal_for_all_users, item_filter_funcs=[], day_filter_funcs=[], user_filter_funcs=[]):
  #sessions_for_user_by_day_and_goal_for_all_users = get_sessions_for_user_by_day_and_goal_for_all_users()
  output = make_dictlist_days(sessions_for_user_by_day_and_goal_for_all_users, item_filter_funcs, day_filter_funcs, user_filter_funcs)
  return convert_list_of_dicts_into_dataframe(output)

# item filter funcs
def enabled_goals_only(x):
  return x['is_goal_enabled'] == True

# day filter funcs
def not_first_day(x):
  return x['days_since_install'] != 0

def not_last_day(x):
  return x['days_until_last'] != 0

# user filter funcs
def at_least_two_days(x):
  return len(x['days_domains_and_sessions']) > 1

def at_least_two_weeks(x):
  #print(len(x))
  return len(x['days_domains_and_sessions']) > 13

df = make_dataframe_days(sessions_for_user_by_day_and_goal_for_all_users, [enabled_goals_only], [not_first_day, not_last_day], [at_least_two_weeks])
#print(make_dataframe_days())

In [97]:
df.dtypes

user                                object
epoch                                int64
domain                              object
time_on_domain_today                 int64
time_on_other_goal_domains_today     int64
is_goal_enabled                       bool
is_goal_frequent                      bool
dtype: object

In [75]:
#ndata = df
#ndata = ndata[ndata['is_goal_enabled'] == True]
#ndata = ndata[ndata['is_goal_frequent'] != 'inconsistent']
#ndata.shape

In [100]:
df

,user,epoch,domain,time_on_domain_today,time_on_other_goal_domains_today,is_goal_enabled,is_goal_frequent
0,c11e5f2d93f249b5083989b2,947,www.youtube.com,233184,859,True,False
1,c11e5f2d93f249b5083989b2,947,www.reddit.com,233184,26,True,False
2,c11e5f2d93f249b5083989b2,953,www.reddit.com,233184,24669,True,True
3,c11e5f2d93f249b5083989b2,953,www.youtube.com,233184,829,True,True
4,c11e5f2d93f249b5083989b2,954,www.reddit.com,233184,0,True,True
5,c11e5f2d93f249b5083989b2,955,www.youtube.com,233184,1351,True,False
6,c11e5f2d93f249b5083989b2,955,www.reddit.com,233184,949,True,False
7,c11e5f2d93f249b5083989b2,956,www.youtube.com,233184,1540,True,False
8,c11e5f2d93f249b5083989b2,956,www.reddit.com,233184,2163,True,False
9,c11e5f2d93f249b5083989b2,957,www.youtube.com,233184,501,True,False


In [101]:
%Rpush df

#filename = 'browser_time_on_domains_sept18.csv'
#df = pd.read_csv(filename)

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [102]:
%%R -w 5 -h 5 --units in -r 200

#install.packages('ez')
#install.packages('lme4')

library(lme4)
library(sjPlot)
library(lmerTest)
#library(ez)

In [105]:
%%R -w 5 -h 5 --units in -r 200

ndata = df
ndata$domain = as.factor(ndata$domain)
ndata$user = as.factor(ndata$user)
ndata$is_goal_enabled = as.factor(ndata$is_goal_enabled)
ndata$is_goal_frequent = as.factor(ndata$is_goal_frequent)
ndata$log_time_on_domain_today = log(ndata$time_on_domain_today)
#ndata = subset(ndata, domain == 'www.facebook.com' | domain == 'www.youtube.com')
summary(ndata)

                       user           epoch                    domain    
 c765fb71d498eb3846f03cd9:  163   Min.   :947.0   www.facebook.com:7635  
 370c2e73147e37d2db03e788:  126   1st Qu.:963.0   www.youtube.com :6359  
 17e7de4f422a37ce6b911178:  118   Median :971.0   twitter.com     :2207  
 37615573eea8075a109f0bef:  116   Mean   :970.1   www.reddit.com  :1396  
 1a15ab8c5d0ef03d6e813dc1:  111   3rd Qu.:978.0   mail.google.com :1384  
 8003c4d51219e6c8d11e4502:  108   Max.   :986.0   www.amazon.com  : 844  
 (Other)                 :19629                   (Other)         : 546  
 time_on_domain_today time_on_other_goal_domains_today is_goal_enabled
 Min.   :        3    Min.   :        0                TRUE:20371     
 1st Qu.:   107397    1st Qu.:       12                               
 Median :   428112    Median :     1506                               
 Mean   :  2008233    Mean   :    49271                               
 3rd Qu.:  1420252    3rd Qu.:     9672              

In [107]:
%%R -w 5 -h 5 --units in -r 200

ndata = df
subset(ndata, time_on_domain_today == 105384637)

                         user epoch           domain time_on_domain_today
8403 6216c587462fb5fa9ca9f0e9   955  www.youtube.com            105384637
8404 6216c587462fb5fa9ca9f0e9   956  www.youtube.com            105384637
8405 6216c587462fb5fa9ca9f0e9   956 www.facebook.com            105384637
8406 6216c587462fb5fa9ca9f0e9   957  www.youtube.com            105384637
8407 6216c587462fb5fa9ca9f0e9   957 www.facebook.com            105384637
8408 6216c587462fb5fa9ca9f0e9   958 www.facebook.com            105384637
8409 6216c587462fb5fa9ca9f0e9   958  www.youtube.com            105384637
8410 6216c587462fb5fa9ca9f0e9   959  www.youtube.com            105384637
8411 6216c587462fb5fa9ca9f0e9   962 www.facebook.com            105384637
8412 6216c587462fb5fa9ca9f0e9   962  www.youtube.com            105384637
8413 6216c587462fb5fa9ca9f0e9   963  www.youtube.com            105384637
8414 6216c587462fb5fa9ca9f0e9   963 www.facebook.com            105384637
8415 6216c587462fb5fa9ca9f0e9   964  w

In [106]:
%%R -w 5 -h 5 --units in -r 200

#results <- lmer(time_on_domain_today ~ is_goal_frequent + (time_on_other_goal_domains_today) + (1|user) + (1|domain) , data = ndata)
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (1|user) + (1|domain) , data = ndata)
#results <- lmer(time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|domain) , data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lmer(log_time_on_domain_today ~ is_goal_frequent + (1|user), data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
#results <- lm(log_time_on_domain_today ~ is_goal_frequent, data = ndata)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: log_time_on_domain_today ~ is_goal_frequent + (1 | domain)
   Data: ndata

REML criterion at convergence: 82419.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.5835 -0.6871  0.0082  0.6932  3.1920 

Random effects:
 Groups   Name        Variance Std.Dev.
 domain   (Intercept) 0.5397   0.7347  
 Residual             3.3387   1.8272  
Number of obs: 20371, groups:  domain, 13

Fixed effects:
                       Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)           1.287e+01  2.369e-01  2.115e+00  54.342 0.000231 ***
is_goal_frequentTRUE -8.175e-03  2.563e-02  2.034e+04  -0.319 0.749782    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
is_gl_fTRUE -0.054


In [49]:
#print(df)